In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
base_url = "https://www.pisos.com/venta/pisos-madrid/"
urls_pisos = [] 

i = 1  
max_pisos = 10  

while len(urls_pisos) < max_pisos:
    url = f"{base_url}{i}/"
    response = requests.get(url)

    if response.status_code != 200:
        print(f'Página {i} no encontrada, deteniendo el scraping.')
        break

    soup = BeautifulSoup(response.text, "html.parser")
    anuncios = soup.find_all("a", class_="ad-preview__title")

    for anuncio in anuncios:
        piso_url = anuncio.get("href")  
        if piso_url:
            piso_url_completa = f'https://www.pisos.com{piso_url}'  
            urls_pisos.append(piso_url_completa)

            if len(urls_pisos) >= max_pisos:
                break

    print(f'Página {i} procesada. Total URLs extraídas: {len(urls_pisos)}')

    i += 1  
    time.sleep(1)  

In [ ]:
pisos_lista = []

for piso_url in urls_pisos:
    piso_response = requests.get(piso_url)

    if piso_response.status_code != 200:
        print(f"Error al cargar {piso_url}")
        continue

    piso_soup = BeautifulSoup(piso_response.text, "html.parser")

    try:
       
        nombre = piso_soup.find("div", class_="ad-preview__title")
        nombre = nombre.text.strip() if nombre else "No disponible"

        
        precio = piso_soup.find("div", class_="price__value jsPriceValue")
        precio = precio.text.strip() if precio else "No disponible"

        
        ubicacion = piso_soup.find("div", class_="p-sm ad-preview__subtitle")
        ubicacion = ubicacion.text.strip() if ubicacion else "No disponible"

        
        caracteristicas_section = piso_soup.find("div", class_="features-container")

        
        superficie = habitaciones = baños = planta = tipo_casa = conservacion = referencia = "No disponible"

        if caracteristicas_section:
            caracteristicas_items = caracteristicas_section.find_all("div", class_="features__feature")

            for item in caracteristicas_items:
                label = item.find("span", class_="features__label")
                value = item.find("span", class_="features__value")

                if label and value:
                    texto_label = label.text.strip()
                    texto_value = value.text.strip()

                    if "Superficie construida" in texto_label:
                        superficie = texto_value
                    elif "Habitaciones" in texto_label:
                        habitaciones = texto_value
                    elif "Baños" in texto_label:
                        baños = texto_value
                    elif "Planta" in texto_label:
                        planta = texto_value
                    elif "Tipo de casa" in texto_label:
                        tipo_casa = texto_value
                    elif "Conservación" in texto_label:
                        conservacion = texto_value
                    elif "Referencia" in texto_label:
                        referencia = texto_value

        
        pisos_lista.append({
            "Nombre": nombre,
            "Precio": precio,
            "Ubicación": ubicacion,
            "Superficie (m²)": superficie,
            "Habitaciones": habitaciones,
            "Baños": baños,
            "Planta": planta,
            "Tipo de Casa": tipo_casa,
            "Conservación": conservacion,
            "Referencia": referencia,
            "URL": piso_url
        })

    except Exception as e:
        print(f"Error extrayendo datos de {piso_url}: {e}")

    time.sleep(1)  



df = pd.DataFrame(pisos_lista)

In [ ]:
df